In [ ]:
import os
import pandas as pd
import re
import json
import openai
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate

from values import Values
import warnings

warnings.filterwarnings('ignore')
val = Values()

In [ ]:
openai.api_key = val.openai
os.environ['OPENAI_API_KEY'] = val.openai

wg_list = val.wr_gr
wg_name = val.wr_name

In [ ]:
## loading the csv file that has the product information for each warengruppe (previously generated using wr_folder_building.ipynb)
file = pd.read_csv(f'{val.parent_dir}{wg_list}/{wg_list}.csv',delimiter=';',encoding='utf-8')
mined_text = file.copy()

In [ ]:
## Extracting the sets from the items to process them later
mined_text['TEILIG'] = mined_text['BESCHREIBUNG'].str.extract(r'( *\d+ *-*tlg.|\d+-teilig|\d+tlg.*|\d+-*er *-*set|\d+ set|\d+ ?ply)',flags=re.IGNORECASE)
mined_text = mined_text[mined_text['TEILIG'].isna() == True]
mined_text = mined_text[['NUMMER','NAME','BESCHREIBUNG']]
mined_text = mined_text[:30]

In [ ]:
allowed_keys = [
    "NUMMER", # Produkt ID
    "NAME", # 1-Stufen Sicherheitsleiter
    "MATERIAL", # Aluminium, Kunststoff
    "PRODUKTART", # Wasserkocher, Mobiler Grillwagen
    "FARBE-TEXT", # Rot
    "GROESSE-TEXT", # klein, groß
    "HERGESTELLT", # Deutschland
    "SET", # 3-tlg.
    "STUECK", # 1 Stück
    "DESIGN", # Extrabreite Stufen mit Antirutschprofil
    "MOTIV", # Blumenmotiv
    "MUSTER", # großer Pfingstrosen-Print
    "BREITE", # 31,5 cm (obere Standfläche)
    "TIEFE", # 14,5 cm (Trittstufen)
    "HOEHE", # 23 cm (obere Standfläche)
    "DURCHMESSER", # Ø 5,5 cm 
    "BODEN", # Ø 25,5 cm 
    "ANDERE-ABMESSUNGEN", # [ 10 x 15 x 5 cm (B/T/H) ]
    "KLINGE-LAENGE", # 15 cm
    "GESAMT-LAENGE", # 25 cm
    "STAERKE", # 3 mm
    "MENGE", # 4
    "GESCHMACK", # cremiger Haselnuss-Geschmack
    "ZUTATEN", # Rindfleisch, Wasser, Salz
    "VERWENDUNGSZWECK", # Zu Pasta, Pizza oder Calzone
    "LAGERUNGSHINWEIS", # Kühl und Trocken lagern
    "VERPACKUNGSGROESSE", # Enthält 6 Cannoli
    "INHALT", # 3 x 200 ml
    "HALTBARKEIT", # 18 Monate haltbar
    "FREI-VON", # Farbstoffen, Konservierungsstoffen
    "GEWICHT", # 15 kg
    "VOLUMEN", # 1 l
    "TEMPERATUR", # 100 °C
    "VOLT", # 230V
    "WATT", # 1.000W
    "DREHZAHL", # bis zu 28.000 Umdrehungen/Minute
    "KABEL", # 1 m
    "LED", # LED Anzeige
    "OFENFEST", # Ofenfest bis 120 °C, Grillfest, Backofenfest
    "GRIFF", # mit Holzgriff
    "DECKEL", # mit Deckel
    "FORM", # Rund
    "KRATZFEST", # kratzfest emailliertem
    "INDUCKTION", # für alle Herdarten inklusive Induktion
    "KERN", # Gusseisenkern
    "ANTIHAFTBESCHICHTUNG", # 3-fache, Hartanodisieren, Antihaftversiegelung
    "PFANNENSTRUKTUR", # Wabenstruktur, Netzbodenstruktur, Rillenstruktur
    "PFANNENART", # Servierpfanne
    "GRILLART", # Tischgrill, Elektrogrill, Gasgrill
    "MASCHINENWASCHBAR", # Spülmaschinfest bis 60 °C
    "PFLEGEHINWEIS", # Fleckschutz lässt sich durch Bügeln auffrischen
    "RUTSCHFEST", # Rutschfest
    "ROSTFREI", # Rostfrei
    "SCHUTZ", # Teflon (TM)-Ausrüstung , Fleckenschutz
    "BRAND", # Hagen Grote
    "QUALITAET", # Hochfeste Aluminiumstruktur
    "ZERTIFIKATE", # OEKO-TEX® STANDARD 100, BIO
    "ANWENDUNG", # Sicherheitsleiter
    "ANDERE-EIGENSCHAFTEN" # ["Standsicher", "Superleicht", "Ultraflach", "Fester Stand", "Oberer Haltebügel", "Geprüfte Sicherheit"]
]

In [ ]:
uncleaned_data = []

for id,name, beschreibung in zip(mined_text['NUMMER'],mined_text['NAME'],mined_text['BESCHREIBUNG']):


    # Define a prompt template
    prompt_template = f"""
                        [INST]
                        1. Extrahieren Sie die Produktattribute aus der folgenden Beschreibung und geben Sie sie als gültiges JSON-Objekt in deutscher Sprache aus.
                        2. Verwenden Sie NUR die allowed_keys = {allowed_keys} Schlüssel und ändern Sie diese nicht.
                        3. Nutzen Sie ALLE erlaubten Schlüssel im JSON-Objekt und setzen Sie nicht verfügbare Werte als "N/A".
                        4. Beschränken Sie die Werte auf maximal 50 Zeichen.
                        5. Verwenden Sie NUR die im Text vorhandenen Informationen, ohne Schlüsse oder Vermutungen.
                        6. Das JSON-Objekt sollte KEINE ASCII-Kodierung enthalten.
                        7. Überprüfen Sie doppelt, ob die Schlüssel alle genau gleich als "allowed_keys" sind.
                        8. Achten Sie darauf, dass die Reihenfolge der Dimensionen B/T/H ist (Breite, Tiefe, Höhe).
                        9. Werte sollten eindeutige Wörter sein und keine "Ja", "Nein", "True" oder "False". Beispiel: DECKEL = "Mit Deckel".
                        10. Die Ausgabe sollte NUR das JSON-Objekt sein, ohne zusätzlichen Text außerhalb von geschweiften Klammern.
                        11. Verwenden Sie ausschließlich Informationen aus dem aktuellen Text, ohne Vorwissen oder Ableitungen.
                        [/INST]
                        NUMMER: {id}
                        NAME: {name}
                        Beschreibung: {beschreibung}
                        """

    # Create a LangChain prompt
    prompt = PromptTemplate(template=prompt_template, input_variables=["description", "allowed_keys", "id", "name"])

    # Initialize the OpenAI LLM
    llm = ChatOpenAI(model="gpt-4o")  # Ensure this is the correct engine name

    # Initialize the LLMChain with the LLM and the prompt
    llm_chain = LLMChain(llm=llm, prompt=prompt)



    response = llm_chain.run(description=beschreibung,name=name,id=id,allowed_keys=allowed_keys)
    uncleaned_data.append(response)

In [ ]:
## function to clean responses from the LLM
def clean_response(response_text):

    try:
        response_dict = json.loads(response_text)
        cleaned_dict = {key: value.strip() if isinstance(value, str) else value for key, value in response_dict.items()}
        return cleaned_dict

    except json.JSONDecodeError :

        print(f"Error: The response is not a valid JSON object.")
        print(response_text)

        return None

In [ ]:
## cleaning one step further and appending responses to a new list
new_data = []
for id,item in enumerate(uncleaned_data):
    # print(item)
    item = item.replace('```',"")
    item = item.replace('json',"")

    new_data.append(clean_response(item))
    
print(new_data)

In [ ]:
## saving clean json-objects to a json file
with open(f'{val.parent_dir}{val.wr_gr}/{val.wr_name}_{val.wr_gr}.json', 'w', encoding="utf-8") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)    